(vis-narrative)=
# Narrative Data Visualisation

## Introduction

In this chapter, we'll examine the tricks and techniques of *narrative data visualisation*. This type of visualisation gets a lot more press than others; literally, because journalists use it in their work. There are many books written on it too. 

It can be useful to bear in mind the three broad categories of visualisation that are out there:

- exploratory

- scientific

- narrative

For all of these, keep this in mind:

> The purpose of visualisation is insight, not pictures

—Ben Shneiderman, populariser of the highlighted text link

The first of the three kinds of vis, *exploratory visualisation*, is the kind that you do when you're looking and data and trying to understand it. Just plotting the data is a really good strategy for getting a feel for any issues there might be. This is perhaps most famously demonstrated by *Anscombe's quartet*: four different datasets with the same mean, standard deviation, and correlation but *very* different data distributions.


(First let's import the packages we'll need:)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates

# Set seed for reproducibility
np.random.seed(10)
# Set max rows displayed for readability
pd.set_option("display.max_rows", 6)
# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)

 *Anscombe's quartet*:

In [ ]:
x = [10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5]
y1 = [8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68]
y2 = [9.14, 8.14, 8.74, 8.77, 9.26, 8.10, 6.13, 3.10, 9.13, 7.26, 4.74]
y3 = [7.46, 6.77, 12.74, 7.11, 7.81, 8.84, 6.08, 5.39, 8.15, 6.42, 5.73]
x4 = [8, 8, 8, 8, 8, 8, 8, 19, 8, 8, 8]
y4 = [6.58, 5.76, 7.71, 8.84, 8.47, 7.04, 5.25, 12.50, 5.56, 7.91, 6.89]

datasets = {"I": (x, y1), "II": (x, y2), "III": (x, y3), "IV": (x4, y4)}

fig, axs = plt.subplots(
    2,
    2,
    sharex=True,
    sharey=True,
    figsize=(4, 4),
    gridspec_kw={"wspace": 0.08, "hspace": 0.08},
)
axs[0, 0].set(xlim=(0, 20), ylim=(2, 14))
axs[0, 0].set(xticks=(0, 10, 20), yticks=(4, 8, 12))

for ax, (label, (x, y)) in zip(axs.flat, datasets.items()):
    ax.text(0.1, 0.9, label, fontsize=20, transform=ax.transAxes, va="top")
    ax.tick_params(direction="in", top=True, right=True)
    ax.plot(x, y, "o")

    # linear regression
    p1, p0 = np.polyfit(x, y, deg=1)  # slope, intercept
    ax.axline(xy1=(0, p0), slope=p1, color="r", lw=2)

    # add text box for the statistics
    stats = (
        f"$\\mu$ = {np.mean(y):.2f}\n"
        f"$\\sigma$ = {np.std(y):.2f}\n"
        f"$r$ = {np.corrcoef(x, y)[0][1]:.2f}"
    )
    bbox = dict(boxstyle="round", fc="blanchedalmond", ec="orange", alpha=0.5)
    ax.text(
        0.95,
        0.07,
        stats,
        fontsize=9,
        bbox=bbox,
        transform=ax.transAxes,
        horizontalalignment="right",
    )

plt.suptitle("Anscombe's Quartet")
plt.show()

Exploratory visualisation is usually quick and dirty, and flexible too. Some exploratory data viz can be automated, and some of the packages we saw in the chapter on {ref}`exploratory-data-analysis` can do this. For an EDA package that's explicitly built with visalisation in mind, check out [SweetViz](https://github.com/fbdesignpro/sweetviz). Beyond you and perhaps your co-authors/collaborators, not many other people should be seeing your exploratory visualisation.

The second kind, scientific visualisation, is the prime cut of your exploratory visualisation. It's the kind of plot you might include in a more technical paper, the picture that says a thousand words. I often think of the first image of a black hole {cite}`akiyama2019first` as a prime example of this. You can get away with having a high density of information in a scientific plot and, in short format journals, you may need to. The journal *Physical Review Letters*, which has an 8 page limit, has a classic of this genre in more or less every issue. Ensuring that important values can be accurately read from the plot is especially important in these kinds of charts. But they can also be the kind of plot that presents the killer results in a study; they might not be exciting to people who don't look at charts for a living, but they might be exciting and, just as importantly, understandable by your peers.

The third and final kind is narrative visualisation. This is the one that requires the most thought in the step where you go from the first view to the end product. It's a visualisation that doesn't just show a picture, but gives an insight. These are the kind of visualisations that you might see in the *Financial Times*, *The Economist*, or on the BBC News website. They come with aids that help the viewer focus on the aspects that the creator wanted them to (you can think of these aids or focuses as doing for visualisation what bold font does for text). They're well worth using in your work, especially if you're trying to communicate a particular narrative, and especially if the people you're communicating with don't have deep knowledge of the topic. You might use them in a paper that you hope will have a wide readership, in a blog post summarising your work, or in a report intended for a policymaker.

## Narrative Data Visualisation

As discussed, the name of the game here is to communicate a particular narrative. Let's see what tricks we can use to help do this. The three stages of this are: deciding the story, deciding the chart, and creating a chart that helps deliver that narrative.

### Deciding the story

We are only human, and a digestible narrative goes a lot further than lots of data with no thread running through. If you are doing narrative visualisation, you must first be clear about the story you want to tell and why it's important. Let's say you are making a visualisation on topic 'Y', then some reasons why 'Y' might be important that you may want to think about when creating your narrative are:

- Y matters: when Y rises or falls, people are hurt or helped.
- Y is puzzling: it defies easy explanation.
- Y is controversial or political: some argue one thing while others say another.
- Y is big (like the service sector) or common (like traffic jams).
- Y helps someone do something they could not do before.

If you can identify why a story is important, you're half way to designing a narrative visualisation that brings the story into focus. Later on, we'll recreate a chart from the *Financial Times*. In that case, 'Y' is a high for air pollution in Beijing that hurts people (it's also political because of efforts to tackle the problem). So, in this case, the creator of the narrative needs to convey that the pollution has hit a high (presumably relative to other points in time) and that this high is far above safe levels.

### What plot should I use?

Once you know what story you want to tell, you need the right kind of chart for the job. Resources like the *Financial Times*' [visual vocabulary](http://ft-interactive.github.io/visual-vocabulary/) are extremely useful here. You need to ask yourself what element you're trying to highlight: a point in time, the size relative to other units cross-sectionally, the distribution either in numbers or spatially, the difference between groups, how something has changed, etc.? There are charts that can help with all of these and it's well worth looking at the link to get a sense. You can find code for many of the plots that are featured in the {ref}`vis-common-plots` chapter.

### Drawing Attention to Enhance Narrative Visualisation

According to data visualisation master Jon Schwabish's book *Better Data Visusalizations* {cite}`schwabish2021better`, there are 15 ways to draw an audience's attention in a chart:

1. Shape
2. Enclosure
3. Line width
4. Saturation
5. Colour
6. Size
7. Markings
8. Orientation
9. Position
10. Sharpness
11. Length
12. 3D
13. Curvature
14. Density
15. Closure

Sometimes people add a 16th entry to this list, Connection.

![Preattentive Visual Processing, by @jschwabish.](https://github.com/aeturrell/coding-for-economists/raw/main/img/preattentive.png) The different ways to trigger preattentive visual processing, by @jschwabish.

But be warned: not all of these are equivalent! It's much easier to perceive differences in length than it is differences in, say, volume. So if you want your audience to be able to make comparisons or quantitative assessments, you need to pick what techniques you use from this list carefully. Roughly in order of how easy they are to perceive quantitatively, the features are: one common axis, two axes, length, slope, angle, parts of whole, area, volume, saturation, and hue.

### The Final Mile in Narrative Visualisations

Good visualisation helps the viewer to grasp the narrative—rather than leaving them puzzling as to what the key message is. To that end, various adornments may be added to a plot to bring out the narrative. These adornments typically take the form of those we have seen already that draw the eye, but for a successful narrative visualisation they must come together to tell a story.

We'll also make use of some other tricks:

- Text annotations, which can be a useful addition to a chart because they further enhance the narrative.

- Declutter the graph, removing lines that aren't helping frame the story

- Use the title to tell the story, and put the y-axis label horizontally below

- Use faded text for text that isn't contributing directly to the narrative

Let's see an example that brings together all of these elements, recreating a chart from the *Financial Times*: a newspaper that is well-known for its impressive visualisations. The chart tells the story of extremely high levels of air pollution in Beijing at the start of 2021. (Note that the data here disagree with the original Financial Times source, which were unavailable; do not take the numbers too seriously.)

Let's first grab the data:

In [ ]:
df = pd.read_csv(
    "data/beijing_pm.csv",
)
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")
# Restrit to time scale of interest
df = df[(df.index >= "2020-02-28") & (df.index <= "2021-03-01")]
df.head()

Now let's get on to the figure. The code that generates individual parts of the chart is annotated to explain what it's doing.

In [ ]:
# set a level for fading some elements of the plot
fade_alpha = 0.6

fig, ax = plt.subplots()
# Plot a line
ax.plot(df.index, df["pm25"], lw=1.5, color="#12549a")
# Title that gives the narrative
plt.suptitle(
    "Severe dust storm causes sharp rise in Beijing's air pollution",
    size=12,
    ha="left",
    x=0.12,
)
# Horizontal y-axis title, faded
ax.set_title(
    "PM2.5 rolling 7-day average (micrograms per cubic metre)",
    loc="left",
    size=10,
    alpha=fade_alpha,
)
# Time is obvious, so no x-label needed: instead annotate sources
ax.set_xlabel(
    "* Based on annual mean exposure of 10 micrograms per cubic metre \n Source: AQICN",
    loc="left",
    size=6,
    alpha=fade_alpha,
)
# remove chart clutter
for key, spine in ax.spines.items():
    spine.set_visible(False)
ax.tick_params(axis="y", which="both", length=0)
ax.tick_params(axis="x", which="both", color=[1, 0, 0, fade_alpha])
# set aesthetically pleasing limits
ax.set_ylim(0, 200)
ax.set_xlim(None, df.index.max())
# for time series, tick marks on the right help give a sense of right-ward motion
ax.yaxis.tick_right()
# create grid only in y-direction, so viewer can judge level-but with few ticks
ax.yaxis.set_major_locator(ticker.MaxNLocator(4))
ax.grid(which="major", axis="y", lw=0.2)
# add minor ticks for months, x-axis
ax.xaxis.set_minor_locator(mdates.MonthLocator())
# major ticks for quarters, x-axis
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=3))
# label x-axis in Jan 01 format at major ticks
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %y"))
# add an annotation to the "news" here, the latest peak. Use an arrow.
ax.annotate(
    "Worst storm in a decade hits Beijing",
    xy=(df.idxmax(), df.max()),
    xycoords="data",
    xytext=(-30, -40),
    textcoords="offset points",
    ha="right",
    size=9,
    arrowprops=dict(arrowstyle="->", connectionstyle="angle3"),
)
# Add a hatch, with a label, that represents the WHO safe level: this helps
# put the current rise in context
ax.fill_between(
    x=df.index,
    y1=0,
    y2=10,
    hatch="///",
    facecolor="None",
    linewidth=0.1,
    alpha=0.2,
)
ax.annotate(
    "WHO safe level*",
    fontweight="heavy",
    xy=(0.6, 0.01),
    xycoords="axes fraction",
    xytext=(0, 0),
    textcoords="offset points",
    ha="right",
    size=9,
)
# Use the FT background colours
fig.set_facecolor("#fff1e4")
ax.set_facecolor("#fff1e4")
# faded tick labels
ax.tick_params(axis="both", which="both", labelsize=10)
plt.setp(ax.get_xticklabels(), alpha=fade_alpha)
plt.setp(ax.get_yticklabels(), alpha=fade_alpha)
plt.show()

What do we learn from this exercise? Well, there's a big last mile issue with making narrative visualisations. Normally, when coding, you should do your best to avoid hard-coding numbers (for example `x=0.15` above). But in this case getting things just so requires a lot of manual adjustment and that's *after* you've decided what story you're going to tell, and how to show it.

The example above demonstrates many of the pain points of the last mile, like decluttering, adjusting text, careful use of saturation and colour, and ensuring dates are displayed in an aesthetically pleasing way.